## メタプログラミング
プログラムを使ってコードの生成や書き換えなどを行うことをメタプログラミングとする。Julia以外のプログラミング言語では、LispのマクロやPythonのデコレータなどが例になる。

Juliaはメタプログラミングを比較的多く用いる言語である。**メタプログラミングを使用することで、複雑なコードを実行時に生成したり、冗長なコードの反復を回避することができる**。@assertなど、@から始まるマクロの呼び出しから成り立ち、**受け取ったJuliaのコードをオブジェクトや別のコードに変換**する。また、r"\d+"のような正規表現などで知られている非標準文字列リテラルも使われる。これもマクロの一種であり、**受け取った文字列をオブジェクトや別のコードへと変換**する。

### シンボル
シンボルはJuliaが取り扱うデータ型の一種で、処理系の内部で使われる名前(識別子)に対応する。例えば、fooという変数をソースコードで使用すると、そのコードを構文解析したときにfooという名前のシンボルが処理系の内部に作られる。

**名前の前にコロンをつけることで作成することができる**

In [4]:
println(:foo)

typeof(ans)

foo


Symbol

In [5]:
# 以下のようにSymbol型のコンストラクタを呼び出して作ることもできる
Symbol("foo")
# コンストラクタの引数は文字列や数値、他のシンボルを渡して結合した名前を作り出すこともできる
Symbol("foo", :bar, 9)

:foobar9

### 構文木(abstract syntax tree: AST)

入力されたコードやファイルに保存されたコードは単なる文字列だが、コードはJuliaの処理系に読み込まれると構文解析され、抽象構文木と呼ばれるデータ構造に変換される。抽象構文木は単に構文木と呼ばれる。

構文木はJuliaのオブジェクトとして扱うことができる。:(hoge)でJuliaのコードを囲むと、Juliaの処理系はそのコードを実行せずに構文木をオブジェクトとして取り出す。この操作をクォートと呼ぶ

In [7]:
ex = :(2x+1)

:(2x + 1)

一行の単純なコードのクォートは:()を用いるが、複数行からなるコードのクォートにはquoteキーワードを使用する。quoteとendで挟まれた部分のコードは、:(hoge)で囲んだコードと同じようにクォートされる。

整数や文字列などのリテラルを除けば、構文木はExpr型で表現されるオブジェクト型になる(expressionの短縮形)。**Expr型のオブジェクトにはheadフィールドとargsフィールドがあり、headフィールドが構文木の種類を表し、argsフィールドがその構成要素を保持している**。

以下の例では、2x+1という計算式の内部構造をdump関数で観察している。この構文木のheadフィールドは:callというシンボルが収められており、構文木が関数呼び出しであることを意味している。argsフィールドには三つの要素が収められており、1要素目が:+シンボル、2要素目が2xに対応する別の構文木、3要素目が整数1のリテラルである。

In [8]:
dump(ex)

Expr
  head: Symbol call
  args: Array{Any}((3,))
    1: Symbol +
    2: Expr
      head: Symbol call
      args: Array{Any}((3,))
        1: Symbol *
        2: Int64 2
        3: Symbol x
    3: Int64 1


In [9]:
dump(:(x=10))

Expr
  head: Symbol =
  args: Array{Any}((2,))
    1: Symbol x
    2: Int64 10


### 構文木の補間
クォートで作られた構文木には、別の構文木やリテラルを埋め込める。これを**補間(interpolation)**と呼ぶ。**構文は文字列の補間と同じように\$記号を用いる**

$(func(ex))のように関数を適用した結果で補間することもできる

In [12]:
ex = :(3y + 1);
:(2x + $(ex))

:(2x + (3y + 1))

メタプログラミングにおいては、シンボルは変数などの識別子として使われることがほとんどなので、シンボルのリテラルでなく識別子として補間される方が利便性が高い。シンボルのリテラルとして補間する必要がある場合には、以下のようにQuoteNodeでクォートする。

In [13]:
ex = :y;
:(2x + $(ex))

:(2x + y)

In [14]:
:(2x + $(QuoteNode(ex)))

:(2x + :y)